In [1]:
"""https://www.kaggle.com/wspinkaggle/rps-basic-decision-tree"""

from kaggle_environments import make
from kaggle_environments.envs.rps.agents import agents

print(agents)

Loading environment football failed: No module named 'gfootball'
{'rock': <function rock at 0x7f1506fd48b0>, 'paper': <function paper at 0x7f1506ccb820>, 'scissors': <function scissors at 0x7f1506ccb940>, 'copy_opponent': <function copy_opponent at 0x7f1506ccb9d0>, 'reactionary': <function reactionary at 0x7f1506ccba60>, 'counter_reactionary': <function counter_reactionary at 0x7f1506ccbaf0>, 'statistical': <function statistical at 0x7f1506ccbb80>}


In [2]:
# adding random agent
import random

def random_agent(observation, configuration):
    return random.randint(0, configuration.signs-1)

In [3]:
import random

def random_agent(observation, configuration):
    return random.randint(0, configuration.signs-1)

In [4]:
%%writefile submission.py

import random
import numpy as np

from sklearn.model_selection import TimeSeriesSplit
from sklearn.tree import DecisionTreeClassifier

sample_size = 5
opponent_observations = []
my_actions = []

def make_train_set(opponent_observations, my_actions, sample_size):
    observed = np.array([list(x) for x in zip(opponent_observations, my_actions)])
    prediction_sample = observed[-sample_size:].reshape(1, sample_size * 2)

    splits = TimeSeriesSplit(
        max_train_size=sample_size,
        n_splits=len(observed) - sample_size
    )
        
    X = np.array([
            observed[train_index] for train_index, _ in splits.split(observed)
    ])
    
    X = X.reshape(X.shape[0], X.shape[1] * X.shape[2])

    Y = np.array([
            observed[y_index][0][0] for _, y_index in splits.split(observed)
    ])

    Y = Y.reshape(Y.shape[0], 1)
    
    return X, Y, prediction_sample

def action_decision_tree_dev(opponent_observations, my_actions,
    sample_size, configuration=None):

    X, Y, prediction_sample = make_train_set(opponent_observations, my_actions, sample_size)

    model = DecisionTreeClassifier(
        criterion="entropy",
        max_depth = 4,
    )

    model.fit(X, Y)

    prediction = model.predict(prediction_sample)
    
    # action is what beats the opponents prediciton
    action = (prediction + 1) % (configuration.signs if configuration else 3)
    action = int(action[0])
    
    # check if fails on long wait
    import time
    
    return action

def tree_agent(observation, configuration):
    global opponent_observations
    global my_actions
    global sample_size
    
    if observation.step == 0:
        action = 1
        my_actions.append(action)
        return action
    
    elif observation.step <= 2 * sample_size:
        opponent_observations.append(observation.lastOpponentAction)
        action = random.randint(0, configuration.signs-1)
        my_actions.append(action)
        return action
        
    else:
        opponent_observations.append(observation.lastOpponentAction)
        
        action = action_decision_tree_dev(
            opponent_observations,
            my_actions,
            sample_size,
            configuration
        )
        
        my_actions.append(action)
        return action

Writing submission.py


In [5]:
%run -i "submission.py"

In [6]:
# some naive tests

a = action_decision_tree_dev(
    opponent_observations = [0, 1, 1, 1, 2, 2, 0, 1, 1, 1, 2, 2], #opp mirros me
    my_actions = [1, 1, 1, 2, 2, 0, 1, 1, 1, 2, 2, 0],
    sample_size = 5,
    configuration=None
)  #expected outcome: 0

print(a, sep="\n")

1


In [7]:
env = make("rps", configuration={"episodeSteps": 100}, debug=False)

env.run([tree_agent, agents["statistical"]])
env.render(mode="ipython", width=600, height=600)

env = make("rps", configuration={"episodeSteps": 100}, debug=True)

env.run([tree_agent, agents["counter_reactionary"]])
env.render(mode="ipython", width=600, height=600)

In [8]:
import numpy as np
l = len(opponent_observations)

print(f"Opponent Observations: {opponent_observations}",
      f"My Actions: {my_actions[:-1]}",
     sep="\n")

Opponent Observations: [0, 2, 2, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 1, 1, 1, 0, 0, 0, 0, 2, 2, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1]
My Actions: [1, 2, 2, 0, 2, 2, 1, 1, 1, 0, 0, 0, 0, 0, 2, 2, 2, 1, 1, 1, 0, 0, 0, 0, 2, 2, 1, 1, 2, 2, 0, 1, 1, 0, 2, 2, 0, 1, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2]
